# Wikibase 代理

这个笔记本演示了一个使用 sparql 生成的非常简单的 wikibase 代理。虽然此代码旨在可以在任何 wikibase 实例上工作，但我们使用 http://wikidata.org 进行测试。

如果您对 wikibase 和 sparql 感兴趣,欢迎帮助改进这个代理。详情和开放问题参见[这里](https://github.com/donaldziff/langchain-wikibase)。


## 准备工作

### API密钥和其他秘钥

我们使用一个 `.ini` 文件,格式如下:
```
[OPENAI]
OPENAI_API_KEY=xyzzy
[WIKIDATA]
WIKIDATA_USER_AGENT_HEADER=argle-bargle
```

In [1]:
import configparser

config = configparser.ConfigParser()
config.read("./secrets.ini")

['./secrets.ini']

### OpenAI API密钥

除非您修改以下代码以使用其他LLM提供商，否则需要一个OpenAI API密钥。

In [2]:
openai_api_key = config["OPENAI"]["OPENAI_API_KEY"]
import os

os.environ.update({"OPENAI_API_KEY": openai_api_key})

### Wikidata用户代理头

Wikidata政策要求用户代理头。参见https://meta.wikimedia.org/wiki/User-Agent_policy。然而，目前此政策并未严格执行。

In [3]:
wikidata_user_agent_header = (
    None
    if not config.has_section("WIKIDATA")
    else config["WIKIDATA"]["WIKIDATA_USER_AGENT_HEADER"]
)

### 如果需要，启用追踪

In [ ]:
# import os
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_PROJECT"] = "default" # 确保此会话实际存在。

# 工具

为此简单代理提供了三个工具：
* `ItemLookup`：用于查找项目的q编号
* `PropertyLookup`：用于查找属性的p编号
* `SparqlQueryRunner`：用于运行sparql查询

## 项目和属性查找

项目和属性查找在一个方法中实现，使用一个弹性搜索端点。并非所有wikibase实例都有它，但wikidata有，我们将从这里开始。

In [ ]:
def get_nested_value(o: dict, path: list) -> any:
    current = o
    for key in path:
        try:
            current = current[key]
        except KeyError:
            return None
    return current


from typing import Optional

import requests


def vocab_lookup(
    search: str,
    entity_type: str = "item",
    url: str = "https://www.wikidata.org/w/api.php",
    user_agent_header: str = wikidata_user_agent_header,
    srqiprofile: str = None,
) -> Optional[str]:
    headers = {"Accept": "application/json"}
    if wikidata_user_agent_header is not None:
        headers["User-Agent"] = wikidata_user_agent_header

    if entity_type == "item":
        srnamespace = 0
        srqiprofile = "classic_noboostlinks" if srqiprofile is None else srqiprofile
    elif entity_type == "property":
        srnamespace = 120
        srqiprofile = "classic" if srqiprofile is None else srqiprofile
    else:
        raise ValueError("entity_type必须是'property'或'item'")

    params = {
        "action": "query",
        "list": "search",
        "srsearch": search,
        "srnamespace": srnamespace,
        "srlimit": 1,
        "srqiprofile": srqiprofile,
        "srwhat": "text",
        "format": "json",
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        title = get_nested_value(response.json(), ["query", "search", 0, "title"])
        if title is None:
            return f"我找不到任何{entity_type}与'{search'相关。请重新措辞并重试"
        # 如果有前缀，去掉它
        return title.split(":")[-1]
    else:
        return "抱歉，我遇到了一个错误。请重试。"

In [6]:
print(vocab_lookup("Malin 1"))

Q4180017


In [7]:
print(vocab_lookup("instance of", entity_type="property"))

P31


In [8]:
print(vocab_lookup("Ceci n'est pas un q-item"))

I couldn't find any item for 'Ceci n'est pas un q-item'. Please rephrase your request and try again


## Sparql运行器

此工具运行sparql - 默认情况下使用wikidata。

In [ ]:
import json
from typing import Any, Dict, List

import requests


def run_sparql(
    query: str,
    url="https://query.wikidata.org/sparql",
    user_agent_header: str = wikidata_user_agent_header,
) -> List[Dict[str, Any]]:
    headers = {"Accept": "application/json"}
    if wikidata_user_agent_header is not None:
        headers["User-Agent"] = wikidata_user_agent_header

    response = requests.get(
        url, headers=headers, params={"query": query, "format": "json"}
    )

    if response.status_code != 200:
        return "该查询失败。也许您可以尝试一个不同的查询？"
    results = get_nested_value(response.json(), ["results", "bindings"])
    return json.dumps(results)

In [10]:
run_sparql("SELECT (COUNT(?children) as ?count) WHERE { wd:Q1339 wdt:P40 ?children . }")

'[{"count": {"datatype": "http://www.w3.org/2001/XMLSchema#integer", "type": "literal", "value": "20"}}]'

# 代理

## 包装工具

In [11]:
import re
from typing import List, Union

from langchain.agents import (
    AgentExecutor,
    AgentOutputParser,
    LLMSingleActionAgent,
    Tool,
)
from langchain.chains import LLMChain
from langchain.prompts import StringPromptTemplate
from langchain_core.agents import AgentAction, AgentFinish

In [ ]:
# 定义代理可以使用哪些工具来回答用户查询
tools = [
    Tool(
        name="ItemLookup",
        func=(lambda x: vocab_lookup(x, entity_type="item")),
        description="当您需要知道项目的q编号时很有用",
    ),
    Tool(
        name="PropertyLookup",
        func=(lambda x: vocab_lookup(x, entity_type="property")),
        description="当您需要知道属性的p编号时很有用",
    ),
    Tool(
        name="SparqlQueryRunner",
        func=run_sparql,
        description="用于从wikibase获取结果",
    ),
]

## 提示

In [ ]:
# 设置基础模板
template = """
通过运行一个sparql查询来回答以下问题，该查询针对一个wikibase，其中p和q项对您来说完全未知。您需要在生成sparql之前发现p和q项。
不要假设您知道任何概念的p和q项。始终使用工具查找所有p和q项。
在您生成sparql之后，您应该运行它。结果将以json返回。
用自然语言总结json结果。

您可以假设以下前缀：
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>

生成sparql时：
* 尽量避免"count"和"filter"查询
* 永远不要将sparql括在反引号中

您可以使用以下工具：

{tools}

使用以下格式：

问题：您必须提供自然语言答案的输入问题
思考：您应该始终考虑该做什么
行动：要采取的行动，应该是[{tool_names}]之一
行动输入：行动的输入
观察：行动的结果
...（此思考/行动/行动输入/观察可以重复N次）
思考：我现在知道最终答案
最终答案：原始输入问题的最终答案

问题：{input}
{agent_scratchpad}"""

In [ ]:
# 设置一个提示模板
class CustomPromptTemplate(StringPromptTemplate):
    # 使用的模板
    template: str
    # 可用工具列表
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # 获取中间步骤（AgentAction，Observation元组）
        # 以特定方式格式化它们
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\n观察：{observation}\n思考："
        # 将agent_scratchpad变量设置为该值
        kwargs["agent_scratchpad"] = thoughts
        # 从提供的工具列表创建一个工具变量
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in self.tools]
        )
        # 为提供的工具创建一个工具名称列表
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # 这省略了`agent_scratchpad`、`tools`和`tool_names`变量，因为这些是动态生成的
    # 这包括`intermediate_steps`变量，因为这是需要的
    input_variables=["input", "intermediate_steps"],
)

## 输出解析器
这与langchain文档中的内容没有变化

In [ ]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # 检查代理是否应该完成
        if "最终答案：" in llm_output:
            return AgentFinish(
                # 返回值通常总是一个带有单个`output`键的字典
                # 目前不建议尝试其他任何东西 :)
                return_values={"output": llm_output.split("最终答案：")[-1].strip()},
                log=llm_output,
            )
        # 解析出行动和行动输入
        regex = r"行动：(.*?)[\n]*行动输入：[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"无法解析LLM输出：`{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # 返回行动和行动输入
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

In [17]:
output_parser = CustomOutputParser()

## 指定LLM模型

In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)

## 代理和代理执行器

In [ ]:
# LLM链由LLM和一个提示组成
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\n观察："],
    allowed_tools=tool_names,
)

In [21]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

## 运行它！

In [ ]:
# 如果您更喜欢内联追踪，请取消注释此行
# agent_executor.agent.llm_chain.verbose = True

In [ ]:
agent_executor.run("J.S.巴赫有多少个孩子？")



> Entering new AgentExecutor chain...
Thought: I need to find the Q number for J.S. Bach.
Action: ItemLookup
Action Input: J.S. Bach

Observation:Q1339I need to find the P number for children.
Action: PropertyLookup
Action Input: children

Observation:P1971Now I can query the number of children J.S. Bach had.
Action: SparqlQueryRunner
Action Input: SELECT ?children WHERE { wd:Q1339 wdt:P1971 ?children }

Observation:[{"children": {"datatype": "http://www.w3.org/2001/XMLSchema#decimal", "type": "literal", "value": "20"}}]I now know the final answer.
Final Answer: J.S. Bach had 20 children.

> Finished chain.


'J.S. Bach had 20 children.'

In [ ]:
agent_executor.run(
    "哈基姆·奥拉朱旺的Basketball-Reference.com NBA球员ID是什么？"
)



> Entering new AgentExecutor chain...
Thought: To find Hakeem Olajuwon's Basketball-Reference.com NBA player ID, I need to first find his Wikidata item (Q-number) and then query for the relevant property (P-number).
Action: ItemLookup
Action Input: Hakeem Olajuwon

Observation:Q273256Now that I have Hakeem Olajuwon's Wikidata item (Q273256), I need to find the P-number for the Basketball-Reference.com NBA player ID property.
Action: PropertyLookup
Action Input: Basketball-Reference.com NBA player ID

Observation:P2685Now that I have both the Q-number for Hakeem Olajuwon (Q273256) and the P-number for the Basketball-Reference.com NBA player ID property (P2685), I can run a SPARQL query to get the ID value.
Action: SparqlQueryRunner
Action Input: 
SELECT ?playerID WHERE {
  wd:Q273256 wdt:P2685 ?playerID .
}

Observation:[{"playerID": {"type": "literal", "value": "o/olajuha01"}}]I now know the final answer
Final Answer: Hakeem Olajuwon's Basketball-Reference.com NBA player ID is "o/ola

'Hakeem Olajuwon\'s Basketball-Reference.com NBA player ID is "o/olajuha01".'